In [60]:
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point, Polygon, MultiPolygon
from geopandas import GeoDataFrame
from IPython.display import IFrame
from descartes import PolygonPatch
ox.config(log_console=True, use_cache=True)
from statistics import mean
import sys
import os
import folium
from geopy.geocoders import Nominatim
import time
import matplotlib as mpl
import seaborn as sns
from scipy import stats
import pandas.core.algorithms as algos
import math
import heapq

In [28]:
# Define Road Quality Impact function
def QI(Q):
    return 1/(1+math.e**(-10*(Q-0.5)))

In [29]:
def initialize_model(place='None',
                     placename='None',
                     network_type='drive',
                     simplify=True,
                     travel_speed=30,
                     airport_list=None,
                     routing_type='time'):
    """This functions initializes a graph of a specified location and calculates several attributes for it
    
    Parameters:
    ----------------------------------
    place:
        A location name for which the graph is desired used in the query of OSM
    placename:
        If the location query name is too long, placename can be used to create a shorter name for the place
    simplify:
        Boolean; if True, the graph is simplified, creating fewer nodes in the network topology, if False, all nodes are created.
    network_type:
        Network type is the type of network is imported
    travel_speed:
        The specified average travel speed one could drive in a network
    airport_list:
        A list of airport names/addresses that are used in the network as logistical hubs
    routing_type:
        The weight that is used to determine the shortest paths for. Options are 'time', 'flowtime' and 'length'.
    
    Returns:
    -----------------------------------
    Pickled data on disk
    
    
    """
    
    print('Configuring graph is initiated')
    # Get graph and gdf
    iso_graph = ox.graph_from_place(
        place, network_type=network_type, retain_all=True, simplify=simplify)
    if isinstance(place, list):
        gdf = ox.gdf_from_places(place)
    else:
        gdf = ox.gdf_from_place(place)

    # Determine airport node
    geolocator = Nominatim()
    airport_list_of_lists = []
    for airport in airport_list:
        location = geolocator.geocode(airport)
        airport_coord = (location.latitude, location.longitude)
        airport_node = ox.get_nearest_node(
            iso_graph, airport_coord, method='haversine')
        airport_list_of_lists.append([airport, airport_coord, airport_node])
    airport_df = pd.DataFrame(
        data=airport_list_of_lists,
        columns=["Airport_name", "Airport_Coord", "Airport_Node"])

    # Write edges to Excel
    writer = pd.ExcelWriter('FinalModelOutput/Edges_{}.xlsx'.format(placename))
    pd.DataFrame(
        [(u, v, data['name'], data['length'], 1)
         if 'name' in data else (u, v, 'None', data['length'], 1)
         for u, v, k, data in iso_graph.edges(data=True, keys=True)],
        columns=[
            'Origin', 'Destination', 'Straatnaam', 'Straatlengte',
            'RoadQuality'
        ]).to_excel(writer)
    writer.save()

    # Set road quality attribute
    df_roads = pd.read_excel('FinalModelOutput/Edges_{}.xlsx'.format(placename))
    attribute = 'RoadQuality'
    for u, v, k, data in iso_graph.edges(data=True, keys=True):
        data[attribute] = df_roads[(df_roads.Origin == u) & (
            df_roads.Destination == v)].RoadQuality.iloc[0]

    # Set default for lanes on 1
    for u, v, j, data in iso_graph.edges(data=True, keys=True):
        if 'lanes' not in data:
            data['lanes'] = '1'

    # Set travel time for each edge
    # In this process, the speed limit of the road is used.
    # When this information is not available, the average speed as chosen initially is used.
    meters_per_minute = travel_speed * 1000 / 60
    
    for u, v, k, data in iso_graph.edges(data=True, keys=True):
        if 'maxspeed' in data.keys():
            try:
                if isinstance(data['maxspeed'], list):
                    data['time'] = data['length'] / (
                        (int(max(data['maxspeed'])) * 1000 / 60
                         ) * QI(data['RoadQuality']))
                else:
                    data['time'] = data['length'] / (
                        (int(data['maxspeed']) * 1000 / 60
                         ) * QI(data['RoadQuality']))
            except:
                data['time'] = data['length'] / (
                    meters_per_minute * QI(data['RoadQuality']))
        else:
            data['time'] = data['length'] / (
                meters_per_minute * QI(data['RoadQuality']))

    # Determine the flow of the edges
    for u, v, k, data in iso_graph.edges(data=True, keys=True):
        data['flowtime'] = data['time'] / int(max(data['lanes']))

    # Determine edge betweenness centrality SUBSET
    dict_edge_betweenness_centrality_subset = nx.edge_betweenness_centrality_subset(
        iso_graph,
        sources=[
            airport_df.iloc[n].Airport_Node for n in range(len(airport_df))
        ],
        targets=list(iso_graph.nodes()),
        normalized=False,
        weight=routing_type)

    for u, v, k in iso_graph.edges:
        iso_graph.edges[(
            u, v, k
        )]['EdgeBetweennessCentralitySubset'] = dict_edge_betweenness_centrality_subset[(u,
                                                                            v)]

    # Determine edge betweenness centrality OVERALL
    dict_edge_betweenness_centrality = nx.edge_betweenness_centrality(
        iso_graph,
        normalized=False,
        weight=routing_type)

    for u, v, k in iso_graph.edges:
        iso_graph.edges[(
            u, v, k
        )]['EdgeBetweennessCentrality'] = dict_edge_betweenness_centrality[(u,
                                                                            v)]


        
    # Determine node betweenness centrality SUBSET
    bcdicSS = nx.betweenness_centrality_subset(iso_graph, 
                                             sources=[airport_df.iloc[n].Airport_Node for n in range(len(airport_df))], 
                                             targets=list(iso_graph.nodes()), 
                                             normalized=False, weight=routing_type)
    print("bcdicSS is complete...")
    for i in [int(iso_graph.node[i]['osmid']) for i in iso_graph.nodes]:
        iso_graph.node[i]['NodeBetweennessCentralitySubset'] = bcdicSS[i]

        
    # Determine node betweenness centrality OVERALL
    bcdic = nx.betweenness_centrality(iso_graph, normalized=False, weight=routing_type)
    print("bcdic is complete...")
    for i in [int(iso_graph.node[i]['osmid']) for i in iso_graph.nodes]:
        iso_graph.node[i]['NodeBetweennessCentrality'] = bcdic[i]
        
        
    
    


    #     for n in range(len(airport_df)):
    #         print('The airport {} betweenness centrality is'.format(
    #             airport_df.iloc[n].Airport_name), iso_graph.node[airport_df.iloc[
    #                 n].Airport_Node]['NodeBetweennessCentrality'])
    #         print('\nWhile the average betweenness centrality is',
    #               mean(bcdic.values()))

    # Save the graph, gdf, airport_df and such to disk to disk
    nx.write_gpickle(iso_graph,
                     'FinalModelOutput/iso_graph_{}'.format(placename))
    gdf.to_pickle('FinalModelOutput/gdf_{}'.format(placename))
    airport_df.to_pickle(
        'FinalModelOutput/airport_df_{}'.format(placename))

    print("Model is finished with initiating.")
    return iso_graph, gdf, airport_df

In [30]:
def update_model(place='None',
                 placename='None',
                 travel_speed=30,
                 airport_list='None',
                 routing_type='time'):
    """This functions loads the existing graph on disk into Jupyter notebook and updates it with new information that has become available
    
    Parameters:
    ----------------------------------
    place:
        A location name for which the graph is desired used in the query of OSM
    placename:
        If the location query name is too long, placename can be used to create a shorter name for the place
    simplify:
        Boolean; if True, the graph is simplified, creating fewer nodes in the network topology, if False, all nodes are created.
    network_type:
        Network type is the type of network is imported
    travel_speed:
        The specified average travel speed one could drive in a network
    airport_list:
        A list of airport names/addresses that are used in the network as logistical hubs
    routing_type:
        The weight that is used to determine the shortest paths for. Options are 'time', 'flowtime' and 'length'.
    
    
    
    Returns:
    -----------------------------------
    Pickled data on disk
    
    
    """
    print('Updating model initiated')
    # Read graph from disk
    iso_graph = nx.read_gpickle('FinalModelOutput/iso_graph_{}'.format(placename))
    gdf = pd.read_pickle('FinalModelOutput/gdf_{}'.format(placename))
    df_roads = pd.read_excel('FinalModelOutput/Edges_{}.xlsx'.format(placename))
    airport_df = pd.read_pickle(
        'FinalModelOutput/airport_df_{}'.format(placename))

    # Set road quality attribute
    attribute = 'RoadQuality'
    for u, v, k, data in iso_graph.edges(data=True, keys=True):
        data[attribute] = df_roads[(df_roads.Origin == u) & (
            df_roads.Destination == v)].RoadQuality.iloc[0]
 

    # Set travel time for each edge

    # In this process, the speed limit of the road is used.
    # When this information is not available, the average speed as chosen initially is used.
    meters_per_minute = travel_speed * 1000 / 60
    
    for u, v, k, data in iso_graph.edges(data=True, keys=True):
        if 'maxspeed' in data.keys():
            try:
                if isinstance(data['maxspeed'], list):
                    data['time'] = data['length'] / (
                        (int(max(data['maxspeed'])) * 1000 / 60
                         ) * QI(data['RoadQuality']))
                else:
                    data['time'] = data['length'] / (
                        (int(data['maxspeed']) * 1000 / 60
                         ) * QI(data['RoadQuality']))
            except:
                data['time'] = data['length'] / (
                    meters_per_minute * QI(data['RoadQuality']))
        else:
            data['time'] = data['length'] / (
                meters_per_minute * QI(data['RoadQuality']))

    # Determine the flow of the edges
    for u, v, k, data in iso_graph.edges(data=True, keys=True):
        data['flowtime'] = data['time'] / int(max(data['lanes']))        

        
    ####
    # Determine edge betweenness centrality SUBSET
    dict_edge_betweenness_centrality_subset = nx.edge_betweenness_centrality_subset(
        iso_graph,
        sources=[
            airport_df.iloc[n].Airport_Node for n in range(len(airport_df))
        ],
        targets=list(iso_graph.nodes()),
        normalized=False,
        weight=routing_type)

    for u, v, k in iso_graph.edges:
        iso_graph.edges[(
            u, v, k
        )]['EdgeBetweennessCentralitySubset'] = dict_edge_betweenness_centrality_subset[(u,
                                                                            v)]

    # Determine edge betweenness centrality OVERALL
    dict_edge_betweenness_centrality = nx.edge_betweenness_centrality(
        iso_graph,
        normalized=False,
        weight=routing_type)

    for u, v, k in iso_graph.edges:
        iso_graph.edges[(
            u, v, k
        )]['EdgeBetweennessCentrality'] = dict_edge_betweenness_centrality[(u,
                                                                            v)]


        
    # Determine node betweenness centrality SUBSET
    bcdicSS = nx.betweenness_centrality_subset(iso_graph, 
                                             sources=[airport_df.iloc[n].Airport_Node for n in range(len(airport_df))], 
                                             targets=list(iso_graph.nodes()), 
                                             normalized=False, weight=routing_type)
    print("bcdicSS is complete...")
    for i in [int(iso_graph.node[i]['osmid']) for i in iso_graph.nodes]:
        iso_graph.node[i]['NodeBetweennessCentralitySubset'] = bcdicSS[i]

        
    # Determine node betweenness centrality OVERALL
    bcdic = nx.betweenness_centrality(iso_graph, normalized=False, weight=routing_type)
    print("bcdic is complete...")
    for i in [int(iso_graph.node[i]['osmid']) for i in iso_graph.nodes]:
        iso_graph.node[i]['NodeBetweennessCentrality'] = bcdic[i]
        
    ######
        
        

    # Save the graph, gdf, airport_df and such to disk to disk
    nx.write_gpickle(iso_graph,
                     'FinalModelOutput/iso_graph_{}'.format(placename))
    gdf.to_pickle('FinalModelOutput/gdf_{}'.format(placename))
    airport_df.to_pickle(
        'FinalModelOutput/airport_df_{}'.format(placename))

    print("Model is finished with updating.")
    return iso_graph

In [31]:
# def make_iso_polys_per_airport(G,
#                            radius,
#                            edge_buff=50,
#                            node_buff=100,
#                            infill=True,
#                            routing_type='time'):
#     isochrone_polys = []
#     for n in range(len(airport_df)):
#         subgraph = nx.ego_graph(
#             G,
#             airport_df.iloc[n].Airport_Node,
#             radius=radius,
#             distance=routing_type)

#         node_points = [
#             Point((data['x'], data['y']))
#             for node, data in subgraph.nodes(data=True)
#         ]
#         try:
#             nodes_gdf = gpd.GeoDataFrame(
#                 {
#                     'id': subgraph.nodes()
#                 }, geometry=node_points)
#         except KeyError:
#             nodes_gdf = gpd.GeoDataFrame({'id':list(subgraph.nodes())},geometry=node_points)
#         nodes_gdf = nodes_gdf.set_index('id')
#         edge_lines = []
#         for n_fr, n_to in subgraph.edges():
#             f = nodes_gdf.loc[n_fr].geometry
#             t = nodes_gdf.loc[n_to].geometry
#             edge_lines.append(LineString([f, t]))

#         n = nodes_gdf.buffer(node_buff).geometry
#         e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
#         all_gs = list(n) + list(e)
#         new_iso = gpd.GeoSeries(all_gs).unary_union

#         if infill:
#             new_iso = Polygon(new_iso.exterior)
#         isochrone_polys.append(new_iso)
# #     return isochrone_polys

In [32]:
def visualise_model(place,
                    placename,
                    routing_type,
                    colorbar_unit,
                    reachability_radius,
                    cMap='RdYlGn',
                    nbuff=75,
                    ebuff=150,
                    top_level_bc=20,
                    airport_ns=500,
                    top_nbc_size=200,
                    top_ebc_size=10,
                    edge_size_scale_factor=1):
    
    
    def make_iso_polys_per_airport(G,
                           radius,
                           edge_buff=50,
                           node_buff=100,
                           infill=True,
                           routing_type='time'):
        isochrone_polys = []
        for n in range(len(airport_df)):
            subgraph = nx.ego_graph(
                G,
                airport_df.iloc[n].Airport_Node,
                radius=radius,
                distance=routing_type)

            node_points = [
                Point((data['x'], data['y']))
                for node, data in subgraph.nodes(data=True)
            ]
            try:
                nodes_gdf = gpd.GeoDataFrame(
                    {
                        'id': subgraph.nodes()
                    }, geometry=node_points)
            except KeyError:
                nodes_gdf = gpd.GeoDataFrame({'id':list(subgraph.nodes())},geometry=node_points)
            nodes_gdf = nodes_gdf.set_index('id')
            edge_lines = []
            for n_fr, n_to in subgraph.edges():
                f = nodes_gdf.loc[n_fr].geometry
                t = nodes_gdf.loc[n_to].geometry
                edge_lines.append(LineString([f, t]))

            n = nodes_gdf.buffer(node_buff).geometry
            e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
            all_gs = list(n) + list(e)
            new_iso = gpd.GeoSeries(all_gs).unary_union

            if infill:
                new_iso = Polygon(new_iso.exterior)
            isochrone_polys.append(new_iso)
        return isochrone_polys
    
    print("Starting on the reachability visualisation")
    
    plt.clf()
    # Read data from disk
    iso_graph = nx.read_gpickle(
        'FinalModelOutput/iso_graph_{}'.format(placename))
    gdf = pd.read_pickle('FinalModelOutput/gdf_{}'.format(placename))
    airport_df = pd.read_pickle(
        'FinalModelOutput/airport_df_{}'.format(placename))

    # Project graph to UTM, this is necessary for a proper projection of the coordinates
    iso_graph = ox.project_graph(iso_graph)

    # Set color bins according to available radii
    iso_colors = ox.get_colors(
        n=len(reachability_radius), cmap=cMap, return_hex=True)

    # Get the x largest edge/node betweenness centralities
    largest_ebc = heapq.nlargest(top_level_bc, [
        data['EdgeBetweennessCentralitySubset']
        for u, v, k, data in iso_graph.edges(data=True, keys=True)
    ])
    largest_nbc = heapq.nlargest(
        top_level_bc, [
            data['NodeBetweennessCentralitySubset']
            for u, data in iso_graph.nodes(data=True)
        ])

    # set node color and node size
    nc = [
        'blue' if nd in [
            airport_df.iloc[n].Airport_Node for n in range(len(airport_df))
        ] else 'black'
        if data['NodeBetweennessCentralitySubset'] in largest_nbc else 'none'
        for nd, data in iso_graph.nodes(data=True)
    ]
    ns = [
        airport_ns if nd in [
            airport_df.iloc[n].Airport_Node for n in range(len(airport_df))
        ] else top_nbc_size
        if data['NodeBetweennessCentralitySubset'] in largest_nbc else 0
        for nd, data in iso_graph.nodes(data=True)
    ]

    es = [
        top_ebc_size if data['EdgeBetweennessCentralitySubset'] in largest_ebc else 1
        for u, v, j, data in iso_graph.edges(data=True, keys=True)
    ]
    ec = [
        'black' if data['EdgeBetweennessCentralitySubset'] in largest_ebc else 'none'
        for u, v, j, data in iso_graph.edges(data=True, keys=True)
    ]


    # Create the graph: Loop over the airports
    fig, ax = ox.plot_graph(
        iso_graph,
        fig_height=16,
        show=False,
        close=False,
        edge_color=ec,
        edge_alpha=0.4,
        node_color=nc,
        node_size=ns,
        edge_linewidth=es)
    
    i = 0
    for radius in sorted(reachability_radius, reverse=True):
        try:
            sys.stdout.write("\rWorking on number {} out of {}...".format(
                i + 1, len(reachability_radius)))
            sys.stdout.flush()
            isochrone_polys = make_iso_polys_per_airport(
                iso_graph, radius, ebuff, nbuff, True, routing_type)
            for polygon, fc in zip(isochrone_polys, iso_colors):
                patch = PolygonPatch(
                    polygon, fc=iso_colors[i], ec='none', alpha=0.9, zorder=-1)
                ax.add_patch(patch)

            i += 1
        except:
            print("Something went wrong with radius {}".format(radius))
            continue




    #     #Set the shape around the network
    #     for geometry in gdf['geometry'].tolist():
    #         if isinstance(geometry, (Polygon, MultiPolygon)):
    #             if isinstance(geometry, Polygon):
    #                 geometry = MultiPolygon([geometry])
    #             for polygon in geometry:
    #                 patch = PolygonPatch(
    #                     polygon,
    #                     fc='#cccccc',
    #                     ec='k',
    #                     linewidth=3,
    #                     alpha=0.1,
    #                     zorder=-1)
    #                 ax.add_patch(patch)

    #     margin = 0.02
    #     west, south, east, north = gdf.unary_union.bounds
    #     margin_ns = (north - south) * margin
    #     margin_ew = (east - west) * margin
    #     ax.set_ylim((south - margin_ns, north + margin_ns))
    #     ax.set_xlim((west - margin_ew, east + margin_ew))


    # Set colormap legend
    reachability_radius_cmap = reachability_radius
    reachability_radius_cmap.append(0)
    reachability_radius_cmap = set(sorted(reachability_radius_cmap))

    # Position of the bar
    ax_bar = fig.add_axes([0.9, 0.2, 0.03, 0.65])
    cmap = mpl.colors.ListedColormap(iso_colors[::-1])
    bounds = sorted(list(reachability_radius_cmap))
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    cb2 = mpl.colorbar.ColorbarBase(
        ax_bar,
        cmap=cmap,
        norm=norm,
        extendrect=True,
        extend='neither',
        ticks=bounds,
        spacing='uniform',
        orientation='vertical',
    )
    cb2.set_label(
        'Reachability of communities ({})'.format(colorbar_unit), fontsize=20)
    cb2.ax.set_yticklabels(cb2.ax.get_yticklabels(), fontsize=18)
    ax.set(title='{}'.format(placename))
    ax.title.set_fontsize(24)
    print('\nFinished reachability map')
    #fig.savefig('Successful Isochrones/Rotterdam_Thesis_Figures/Isochrones_{}_{}.pdf'.format(placename, routing_type), transparent=False, dpi=200)
    #print('Saved figure for {}, with routing option "{}" '.format(placename, routing_type))

    #-----------------------------------------------------------
    # Road quality visualisation
    sys.stdout.write("\rNow the road quality visualisation is initiated")
    sys.stdout.flush()
    # Set nodecolors and nodesizes
    nc = [
        'blue' if node in [
            airport_df.iloc[n].Airport_Node for n in range(len(airport_df))
        ] else 'black'
        if data['NodeBetweennessCentralitySubset'] in largest_nbc else 'none'
        for node, data in iso_graph.nodes(data=True)
    ]
    ns = [
        airport_ns if node in [
            airport_df.iloc[n].Airport_Node for n in range(len(airport_df))
        ] else top_nbc_size
        if data['NodeBetweennessCentralitySubset'] in largest_nbc else 0
        for node, data in iso_graph.nodes(data=True)
    ]

    # Set Colors for Road qualities
    edge_color_dict = {}
    count = 0
    for i in np.arange(0, 1.1, 0.1):
        edge_color_dict[round(i, 1)] = ox.get_colors(
            11, 'RdYlGn', return_hex=True)[count]
        count += 1

    ec = [
        edge_color_dict[round(i, 1)] for i in [
            data['RoadQuality']
            for u, v, j, data in iso_graph.edges(data=True, keys=True)
        ]
    ]

    es = [
        edge_size_scale_factor*b / 4 for b in pd.qcut(
            [
                data['EdgeBetweennessCentralitySubset']
                for u, v, data in iso_graph.edges(data=True)
            ],
            5,
            retbins=True,
            duplicates='drop',
            labels=list(
                np.arange(1, 1 + (len(
                    pd.unique(
                        pd.qcut(
                            [
                                data['EdgeBetweennessCentralitySubset']
                                for u, v, data in iso_graph.edges(data=True)
                            ],
                            5,
                            retbins=True,
                            duplicates='drop')[0]))))**2))[0]
    ]

    figq, axq = ox.plot_graph(
        iso_graph,
        fig_height=16,
        show=False,
        close=False,
        edge_color=ec,
        edge_alpha=0.8,
        node_zorder=3,
        node_color=nc,
        node_size=ns,
        edge_linewidth=es)

    # # Set the shape around the network
    # for geometry in gdf['geometry'].tolist():
    #     if isinstance(geometry, (Polygon, MultiPolygon)):
    #         if isinstance(geometry, Polygon):
    #             geometry = MultiPolygon([geometry])
    #         for polygon in geometry:
    #             patch = PolygonPatch(
    #                 polygon,
    #                 fc='#cccccc',
    #                 ec='k',
    #                 linewidth=3,
    #                 alpha=0.1,
    #                 zorder=-1)
    #             axq.add_patch(patch)

    # Position of the bar
    axq_bar = figq.add_axes([0.9, 0.2, 0.03, 0.65])
    cmap = mpl.colors.ListedColormap(
        ox.get_colors(10, 'RdYlGn', return_hex=True))
    bounds = list(np.arange(0, 1.1, 0.1))
    norm = mpl.colors.BoundaryNorm(bounds, len(bounds))
    cb3 = mpl.colorbar.ColorbarBase(
        axq_bar,
        cmap=cmap,
        norm=norm,
        extendrect=True,
        extend='neither',
        ticks=bounds,
        spacing='proportional',
        orientation='vertical')
    cb3.set_label('Quality of the road', fontsize=20)
    cb3.ax.set_yticklabels(cb3.ax.get_yticklabels(),fontsize=20)

    # margin = 0.02
    # west, south, east, north = gdf.unary_union.bounds
    # margin_ns = (north - south) * margin
    # margin_ew = (east - west) * margin
    # axq.set_ylim((south - margin_ns, north + margin_ns))
    # axq.set_xlim((west - margin_ew, east + margin_ew))

    axq.set(title=placename)
    axq.title.set_fontsize(24)
    sys.stdout.write("\rRoad Quality Visualisation is finished")
    sys.stdout.flush()
    
    #------------------------------------------------------------------------------------------------------------------
    
    # METAPLOTS
    sys.stdout.write("\rNow initializing meta plots configuration...")
    sys.stdout.flush()
    figA, axA = plt.subplots()
    figB, axB = plt.subplots()
    figC, axC = plt.subplots()

    nbc = []
    for id, data in iso_graph.nodes(data=True):
        nbc.append(data['NodeBetweennessCentrality'])

    #Figure 1
    y = list([node for node in airport_df.Airport_Node])
    y_label = list([node for node in airport_df.Airport_name])
    width = [iso_graph.node[i]['NodeBetweennessCentrality'] for i in y]
    y_label.append('Overall average')
    width.append(mean(nbc))
    y_pos =np.arange(len(y_label))
    rects=axA.barh(y_pos, width, align='center', color="coral")
    # axA.set_yticks(y_pos)
    axA.set_yticklabels('',fontsize=14)
    # axA.set_xticklabels(,fontsize=14)
    axA.set_xlabel('Node betweenness centrality',fontsize=14)
    axA.set_ylabel('Airport names', fontsize=14)
    axA.set(title='Node betweenness centrality of airports')
    axA.title.set_fontsize(20)

    totals=[]
    for i in axA.patches:
        totals.append(i.get_width())
    total=sum(totals)
    counter =0    
    for i in axA.patches:
        axA.text(mean(width)/4, i.get_y()+.38,
                y_label[counter], fontsize=15,color='black')
        counter+=1

    # Figure 2:
    ebc = []
    for u, v, data in iso_graph.edges(data=True):
        ebc.append(data['EdgeBetweennessCentrality'])

    axB.hist(ebc, bins=50, color='green',rwidth=0.8, log=True)
    axB.set(title='Edge betweenness centrality distribution')
    axB.set_ylabel('Number of edges',fontsize=14)
    axB.set_xlabel('Edge betweenness centrality', fontsize=14)
    # axB.set_xticks(,rotation=30)
    axB.title.set_fontsize(20)
    # axB.set_yticklabels(axB.get_yticklabels(), fontsize=18)

    #Figure 3:
    axC.hist(nbc, color='blue',bins=100)
    axC.semilogy()
    axC.set(title='Node betweenness centrality distribution')
    axC.title.set_fontsize(20)
    axC.set_ylabel('Number of edges', fontsize=14)
    axC.set_xlabel('Node betweenness centrality', fontsize=14)
    
    
    plt.show()
    plt.clf()
    sys.stdout.write("\rFinished visualising meta plots")
    sys.stdout.flush()
    fig.savefig('FinalModelOutput/Figures/Isochrones_{}_{}.pdf'.format(placename, routing_type), transparent=False, dpi=200)
    figq.savefig('FinalModelOutput/Figures/RoadQuality{}_{}.pdf'.format(placename, routing_type))
    figA.savefig('FinalModelOutput/Figures/Metadata_airportnbc_{}_{}.pdf'.format(placename, routing_type))
    figB.savefig('FinalModelOutput/Figures/Metadata_ebc_{}_{}.pdf'.format(placename, routing_type))
    figC.savefig('FinalModelOutput/Figures/Metadata_nbc_{}_{}.pdf'.format(placename, routing_type))
    print('Saved figure for {}, with routing option "{}" '.format(placename, routing_type))
    #------------------------------------------------------------
    return

The routing option chosen will determine how the shortest paths in the graph are calculated. Below, the routing option can be selected that basically is an attribute that is used as the weight of the shortest path calculation.
A routing option can be chosen from the following list:
- 'time'        (looking for the route that is the quickest)
- 'length'      (looking for solely the shortest route in distance)
- 'flowtime' ( looking for the route that is both the quickest and has the most capacity)

In [72]:
place = [
    'Southern Highlands, Papua Niugini',
    'Hela, Papua Niugini',
    'Enga, Papua Niugini',
    'Western Highlands, Papua Niugini',
]
placename = 'Papua New Guinea'

airport_list = [
    'Tari Airport , Papua New Guinea', 'Moro Airport, Papua New Guinea',
    'Mendi Airport, Papua New Guinea', 'Mount Hagen Airport, Papua New Guinea',
    'Komo Manda Airport, Papua New Guinea']

In [21]:
# place='Saint-Martin, The Netherlands'
# placename=place
# airport_list=['Princess Juliana International Airport, Saint-Martin', 'Dock Maarten, Sint Maarten']

In [ ]:
routing_type= 'time'

# colorbar_unit = 'meters'
colorbar_unit = 'minutes'
# colorbar_unit = 'minutes per volume unit'

# reachability_radius=[20, 40, 50, 60, 70, 90, 100, 130, 170, 200, 230, 250]
reachability_radius = [5, 10, 12, 15, 18, 20, 25, 30, 35, 45, 60]


network_type='drive'
simplify=True
travel_speed=50
cMap='RdYlGn'
nbuff=300
ebuff=600
top_level_bc=50
airport_ns=800
top_nbc_size=400
top_ebc_size=10
edge_size_scale_factor=3


# iso_graph, gdf, airport_df = initialize_model(place, placename, network_type, simplify, travel_speed,
#                   airport_list, routing_type)

iso_graph = update_model(place, placename, travel_speed,
                 airport_list, routing_type)

visualise_model(place, placename, routing_type, colorbar_unit,
                 reachability_radius, cMap, nbuff, ebuff, top_level_bc,
                 airport_ns, top_nbc_size, top_ebc_size, edge_size_scale_factor)

pd.value_counts([data['RoadQuality'] for u,v,k,data in iso_graph.edges(data=True, keys=True)])

Updating model initiated
bcdicSS is complete...
bcdic is complete...
Model is finished with updating.
Starting on the reachability visualisation
